In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%run ../init_env.py
%run ../config.py

from tools.profiler import profile_resources
from asr import evaluate_transcription
import gc


[NeviseCorrector] Using device: cpu
[Nevise] Loaded SubwordBert model with 239,743,651 parameters
[Nevise] Checkpoint 'model.pth.tar' loaded onto cpu


In [2]:

# result = evaluate_transcription(audio_path=audio_file, ground_truth_path=ground_truth_file, diff=False, print_hype_text=True , print_ref_text=True, model_size="medium" )

# @profile_resources
def run_test():

    # audio_file = "../Data/Training/Farsi/Quran/farsi_secondsource.mp3"
    # ground_truth_file = "../Data/Training/Farsi/Quran/farsi_secondsource_transcript-farsi_translation-hebrew.docx"

    audio_file = "../Data/Training/Farsi/Quran/farsi_thirdsource.mp3"
    ground_truth_file = "../Data/Training/Farsi/Quran/farsi_thirdsource_transcript-farsi_translation-hebrew.docx"

    # audio_file = "../Data/Training/Hebrew/hebrew_firstsource.mp4"
    # ground_truth_file = "../Data/Training/Hebrew/hebrew_firstsource_transcript-hebrew_translation-english.docx"

    return evaluate_transcription(
        audio_path=audio_file,
        ground_truth_path=ground_truth_file,
        diff=False,
        print_hyp=True,
        print_ref=True,
        model_size="medium"
    )

# Now execute:
run_test()

# model medium farsi_secondsource
##only hazm 33%
##only p_norm 30%
##both + nevis 29.23% upto 31%

# model medium farsi_secondsource
##both + nevis 31%


# align_whisper_output(result=result, audio_path=audio_file, ground_truth_path=ground_truth_file, diff=True, print_hype_text=True , print_ref_text=True )


[autoreload of _pydevd_bundle.pydevd_additional_thread_info_regular failed: Traceback (most recent call last):
  File "/Users/pouya/PycharmProjects/SmartSummary/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 325, in check
    superreload(m, reload, self.old_objects)
  File "/Users/pouya/PycharmProjects/SmartSummary/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 621, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/pouya/PycharmProjects/SmartSummary/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 447, in update_generic
    update(a, b)
  File "/Users/pouya/PycharmProjects/SmartSummary/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 399, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pouya/PycharmProjects/SmartSummary/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 444, i

No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...
Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.7.1. Bad things might happen unless you revert torch to 1.x.
Detected language: fa (0.98) in first 30s of audio...
✅ Raw transcript saved to transcripts_json/farsi_thirdsource.json

--- Comparing raw transcription (fa) ---


[Nevise] Original sentence: 'به نام خداوند بخشنلی بخشایشگر، آنچه در آسمان ها و آنچه در زمین هست، همه تسبیح خدا میگوگند، و او شکاست نافذیر و حکیم هست. ایچسانی که ایمان آوردید، چرا صخنی میگوید که عمل نمیکنید؟ نظر خدا بسیار موجب خشم هست که صخنی بگوید که عمل نمیکنید. خداوان شسانی را دوست می دارد که در راه او پیکار می کنند. گوی بنای آهنینند. به یاد آورید هنگامی را که موسا به قومش گفت ای قوم من چرا مرا آزار می دهید با این که می دانید م

TypeError: expected string or bytes-like object, got 'dict'

/Users/pouya/PycharmProjects/SmartSummary/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py:247: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if not hasattr(module, "__file__") or module.__file__ is None:
